#インストール

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install transformers==4.18.0 fugashi==1.1.0 ipadic==1.0.0 pytorch-lightning==2.4.0
#!pip install iterative-stratification
!pip install emoji
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.9/336.9 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 105.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 98.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 110.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 927.3/927.3 kB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 60.6 MB/s eta 0:00:00
  Created wheel for fugashi: filename=fugashi-1.1.0-cp310-cp310-linux_x86_64.whl size=257489 sha256=4a364b66dbebe7c4e0f719c05cd6d41d85cb0071ccbcab13174b33fcaa9a2eb5
  Stored in directory: /root/.cache/pip/wheels/f6/bc/9b/6d38a64c5bea6582a87574f5c63b6

# 設定

In [ ]:
focal_loss = True

#データセット用意

In [ ]:
import pandas as pd
import emoji
#レビューリストとラベルリストを返す関数
def csv_to_list(csv_list : list):
  """
  csv_list => [タイトル, レビュー, カテゴリー, 星数, 参考になった数, 生成元ID]
  生成元IDはaug_csv系のみ
  """
  review_list,label_list = [],[]
  label_num = 8 #分類カテゴリ数
  each_label_num = {} #各label数のカウント
  #each_label_numのvalueを初期化
  for i in range(label_num):
    each_label_num[i+1] = 0
  for i in range(len(csv_list)):
    review = emoji.replace_emoji(csv_list[i][1])
    review_list.append(review)
    #csv_list[i][2]がintではなかったら
    if not isinstance(csv_list[i][2], int):
      str_category_list = csv_list[i][2].split(',')
    else:
      str_category_list = [csv_list[i][2]]
    try:
      category_list = [int(num) for num in str_category_list]
    except ValueError:
      print(i)
    onehot_lavel = []
    #空のマルチラベルリスト作成
    for i in range(label_num):
      onehot_lavel.append(0)
    for category in category_list:
      onehot_lavel[int(category)-1] = 1
      each_label_num[int(category)] =  each_label_num[int(category)] + 1
    label_list.append(onehot_lavel)
  return review_list, label_list, each_label_num

def get_aug_origin_ID(csv_list : list):
  aug_origin_ID = []
  for i in range(len(csv_list)):
    aug_origin_ID.append(csv_list[i][5])
  return aug_origin_ID

#シングルラベル分類用に上記のデータを整形
def multi_label_to_single_label(multi_label_list, target_label):
  single_label_list = []
  for i, _label in enumerate(multi_label_list):
    single_label_list.append(multi_label_list[i][target_label - 1])
  return single_label_list

#[0,1,1,0...0]の様なリストからtarget(0 or 1)であるリストのインデックスを返す関数
def get_label_index(label_list, target_label):
  label_index_list = []
  for i, _label in enumerate(label_list):
    if _label == target_label:
      label_index_list.append(i)
  return label_index_list


#インデックスリストを元にreviewsとlabelsの両方のリストから抽出したリストを返す関数
def get_review_label_from_index(label_index_list, reviews, labels):
  return [reviews[i] for i in label_index_list], [labels[i] for i in label_index_list]

# CSVファイルの読み込み
data = pd.read_csv('drive/MyDrive/review/review_ver20250108.csv')
review_label_list = data.values.tolist()
#オーグメンテーション用
data = pd.read_csv('drive/MyDrive/review/aug_review_label2-m2.csv')
aug_review_label_list = data.values.tolist()
"""
#シングルラベル用
data = pd.read_csv('drive/MyDrive/single_label_review.csv')
single_label_review_label_list = data.values.tolist()
"""
review_list,label_list, each_label_num = csv_to_list(review_label_list)
aug_review_list, aug_label_list, aug_each_label_num = csv_to_list(aug_review_label_list)
aug_origin_ID = get_aug_origin_ID(aug_review_label_list)
#single_label_review_list, single_label_label_list, single_label_each_label_num = csv_to_list(single_label_review_label_list)

single_label_target_label = 2
single_label_list = multi_label_to_single_label(label_list, single_label_target_label)
aug_single_label_list = multi_label_to_single_label(aug_label_list, single_label_target_label)

#重み

In [ ]:
print(f'target_label : {single_label_target_label}')
target_weights = len(review_list) / (each_label_num[single_label_target_label] )
print(f'target_weights : {target_weights}')
target_alpha = 1/target_weights
print(f'alpha : {target_alpha}')
gamma = 2
print(f'gamma : {gamma}')

target_label : 2
target_weights : 7.737373737373737
alpha : 0.12924281984334204
gamma : 2


#pytorch定義

In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import BertJapaneseTokenizer, BertForSequenceClassification, BertModel
import pytorch_lightning as pl

import torch.nn.functional as F
from sklearn.metrics import f1_score

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FocalLoss(nn.Module):
    def __init__(self, alpha, gamma, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        BCE_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1 - pt) ** self.gamma * BCE_loss

        if self.reduction == 'mean':
            return F_loss.mean()
        elif self.reduction == 'sum':
            return F_loss.sum()
        else:
            return F_loss

In [ ]:
class BertForSequenceClassification_pl(pl.LightningModule):
    def __init__(self, model_name, num_labels, lr, alpha=target_alpha, gamma=gamma):
        super().__init__()
        self.save_hyperparameters()

        # BERTのロード
        self.bert_sc = BertForSequenceClassification.from_pretrained(
            model_name,
            num_labels=num_labels,
            output_hidden_states=True
        )
        self.test_labels = []
        self.test_labels_predicted = []
        # FocalLoss
        if focal_loss:
            self.loss_fct = FocalLoss(alpha=alpha, gamma=gamma)
        else:
            # グローバル変数使ってるから後で直すかも
            target_weights = len(review_list) / (each_label_num[single_label_target_label] + aug_each_label_num[single_label_target_label])
            class_weights = torch.tensor([1.0, target_weights]).to(self.device)  # デバイスに移動
            self.loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights)

        # 閾値の初期値
        self.threshold = 0.1

    def training_step(self, batch, batch_idx):
        labels = batch.pop('labels')
        batch = {k: v.to(self.device) for k, v in batch.items()}  # バッチをデバイスに移動
        labels = labels.to(self.device)  # ラベルをデバイスに移動
        output = self.bert_sc(**batch)
        logits = output.logits

        if focal_loss:
            loss = self.loss_fct(logits, labels)
        else:
            loss = self.loss_fct(logits.view(-1, self.hparams.num_labels), labels.view(-1))
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        labels = batch.pop('labels')
        batch = {k: v.to(self.device) for k, v in batch.items()}
        labels = labels.to(self.device)

        output = self.bert_sc(**batch)
        logits = output.logits

        probabilities = torch.sigmoid(logits).cpu().detach().numpy()
        labels = labels.cpu().detach().numpy()

        # probabilities の長さを labels に合わせる
        probabilities = probabilities[:len(labels)]
    """
        # F1スコアを最大化する閾値を見つける
        def objective(threshold):
            predictions = (probabilities > threshold).astype(int)
            if predictions.shape != labels.shape:
                predictions = predictions.argmax(axis=1)
            f1 = f1_score(labels, predictions)
            return -f1

        space = [Real(0.1, 0.9, name='threshold')]
        res = gp_minimize(objective, space, n_calls=20, random_state=0)
        self.threshold = res.x[0]

        labels_predicted = (probabilities > self.threshold).astype(int)
        if labels_predicted.shape != labels.shape:
            labels_predicted = labels_predicted.argmax(axis=1)
        num_correct = (labels_predicted == labels).sum()
        accuracy = num_correct / labels.shape[0]
        self.log('val_accuracy', accuracy)

        f1 = f1_score(labels, labels_predicted)
        self.log('val_f1', f1)

        val_loss = self.loss_fct(torch.tensor(probabilities).view(-1, self.hparams.num_labels).to(self.device), torch.tensor(labels).view(-1).to(self.device))  # デバイスに移動
        self.log('val_loss', val_loss)
    """
    def test_step(self, batch, batch_idx):
        labels = batch.pop('labels')
        batch = {k: v.to(self.device) for k, v in batch.items()}  # バッチをデバイスに移動
        labels = labels.to(self.device)  # ラベルをデバイスに移動
        self.test_labels = labels.to('cpu').detach().numpy().copy()
        output = self.bert_sc(**batch)
        logits = output.logits

        probabilities = torch.sigmoid(logits).cpu().detach().numpy()
        labels_predicted = (probabilities > self.threshold).astype(int)
        if labels_predicted.shape != self.test_labels.shape:
            labels_predicted = labels_predicted.argmax(axis=1)
        self.test_labels_predicted = labels_predicted
        print(f'threshold : {self.threshold}')
        print(f'test_labels : {self.test_labels}')
        print(f'predictes_labels : {self.test_labels_predicted}')

        num_correct = (labels_predicted == self.test_labels).sum()
        accuracy = num_correct / self.test_labels.shape[0]
        self.log('accuracy', accuracy)

        f1 = f1_score(self.test_labels, labels_predicted)
        self.log('test_f1', f1)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)

#モデルロード，推論動作確認

In [ ]:
import os
model_path = '/content/drive/My Drive/path_to_save_model/'
model_path = model_path + os.listdir(model_path)[-1]
print(model_path)
# トークナイザーをロード
MODEL_NAME = 'tohoku-nlp/bert-base-japanese-whole-word-masking'
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)
# モデルをロード
model = BertForSequenceClassification_pl.load_from_checkpoint(model_path)
model.bert_sc.save_pretrained('./model_transformers')
bert_sc = BertForSequenceClassification.from_pretrained('./model_transformers')
# モデルの評価モードを設定
bert_sc.eval()

/content/drive/My Drive/path_to_save_model/20250124_05_label2_fold5.ckpt


Downloading:   0%|          | 0.00/252k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/120 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/479 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:349: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_location="cp

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
"""
#推論モードモデルの動作確認

import os

MODEL_NAME = 'tohoku-nlp/bert-base-japanese-whole-word-masking'
# Google Driveに保存したモデルのパス
model_path = '/content/drive/My Drive/path_to_save_model/'
model_path = model_path + os.listdir(model_path)[-1]
print(model_path)

# トークナイザーをロード
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)
# モデルをロード
model = BertForSequenceClassification_pl.load_from_checkpoint(model_path)
model.bert_sc.save_pretrained('./model_transformers')
bert_sc = BertForSequenceClassification.from_pretrained('./model_transformers')
# モデルの評価モードを設定
bert_sc.eval()
# テストデータのトークナイズ
texts = ["2万円を切るお手ごろ価格。たまに訪れる別宅で、在室時だけ電源を入れる、という使い方には十分の性能。継続してつかいつづけるには、さすがに小さくてパワー不足だけど。", "寝室に置いてますが、音も静かで、。アイスやドリンクを入れています。がじゅうぶんな大きさです。"]
inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=128)

# モデルの予測を取得
with torch.no_grad():
    outputs = bert_sc(**inputs)
    predictions = outputs.logits
labels_predicted = (predictions > 0).int().cpu().numpy().tolist()
print(labels_predicted)
# 予測結果の確認
print(predictions)
"""

'\n#推論モードモデルの動作確認\n\nimport os\n\nMODEL_NAME = \'tohoku-nlp/bert-base-japanese-whole-word-masking\'\n# Google Driveに保存したモデルのパス\nmodel_path = \'/content/drive/My Drive/path_to_save_model/\'\nmodel_path = model_path + os.listdir(model_path)[-1]\nprint(model_path)\n\n# トークナイザーをロード\ntokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)\n# モデルをロード\nmodel = BertForSequenceClassification_pl.load_from_checkpoint(model_path)\nmodel.bert_sc.save_pretrained(\'./model_transformers\')\nbert_sc = BertForSequenceClassification.from_pretrained(\'./model_transformers\')\n# モデルの評価モードを設定\nbert_sc.eval()\n# テストデータのトークナイズ\ntexts = ["2万円を切るお手ごろ価格。たまに訪れる別宅で、在室時だけ電源を入れる、という使い方には十分の性能。継続してつかいつづけるには、さすがに小さくてパワー不足だけど。", "寝室に置いてますが、音も静かで、。アイスやドリンクを入れています。がじゅうぶんな大きさです。"]\ninputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=128)\n\n# モデルの予測を取得\nwith torch.no_grad():\n    outputs = bert_sc(**inputs)\n    predictions = outputs.logits\nlabels_predicted = (predictions > 0).i

# データ整形

In [ ]:
import random

def random_sample_sets(aug_review_list, aug_label_list, n):
    """
    ランダムにnセットのレビューと対応するラベルを抽出する関数。

    Args:
        aug_review_list (list): レビューリスト
        aug_label_list (list): レビューに対応するラベルリスト
        n (int): 抽出するセットの数

    Returns:
        tuple: (抽出されたレビューリスト, 抽出されたラベルリスト)
    """
    if len(aug_review_list) != len(aug_label_list):
        raise ValueError("レビューリストとラベルリストの長さが一致しません。")

    if n > len(aug_review_list):
        raise ValueError("nはリストの長さ以下である必要があります。")

    # インデックスをランダムに選択
    indices = random.sample(range(len(aug_review_list)), n)

    # 選択したインデックスに基づいてリストを抽出
    sampled_reviews = [aug_review_list[i] for i in indices]
    sampled_labels = [aug_label_list[i] for i in indices]

    return sampled_reviews, sampled_labels

n = 0
sampled_reviews, sampled_labels = random_sample_sets(aug_review_list, aug_label_list, n)
print("抽出されたレビュー:", sampled_reviews)
print("抽出されたラベル:", sampled_labels)

抽出されたレビュー: []
抽出されたラベル: []


In [ ]:
#変数定義

## 注目ラベルnの設定
n = 2
## 拡張データ数
aug_review_num =256

In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold

labels = np.array([row[n-1] for row in label_list])

# StratifiedKFold を使ってラベル2に基づいて均等に分割
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
fold_indices = []

for train_idx, test_idx in skf.split(np.zeros(len(labels)), labels):
    fold_indices.append((train_idx, test_idx))
    print(test_idx)


[ 11  12  18  19  23  29  34  40  41  43  47  52  54  59  61  68  69  77
  78  79  80  83  84  92  95  98 101 106 111 118 120 122 126 129 135 137
 144 148 151 172 173 178 180 189 191 192 196 198 200 203 215 220 221 223
 224 235 238 240 245 254 260 268 270 272 277 282 284 287 289 297 304 312
 318 328 335 341 342 350 353 356 357 372 376 379 382 387 399 406 407 411
 412 415 425 428 432 444 454 472 474 475 478 488 490 497 498 512 513 533
 535 540 547 552 554 563 570 573 580 582 590 594 595 617 624 628 635 645
 648 650 654 655 658 668 670 675 677 679 684 697 700 703 705 707 711 717
 733 737 750 751 753 758 760 761 762 765]
[  1   4  16  17  21  22  38  39  48  50  55  56  58  64  66  67  71  86
  90  96 104 121 123 128 133 136 143 145 149 152 157 164 179 187 190 193
 194 199 211 222 227 228 229 232 236 250 252 259 261 263 264 266 276 279
 280 281 288 298 303 314 315 319 324 331 333 338 346 347 351 360 365 366
 370 371 378 380 385 390 393 395 400 423 427 430 431 433 434 441 442 446
 456 458 

In [ ]:
#以下に交差検証を行うコード記述
review_list = np.array(review_list)
single_label_list = np.array(single_label_list)
matrix_list,precision_scores, recall_scores = [], [], []
test_results = []
train_texts, test_texts, train_labels, test_labels = [], [], [], []

for i, (train_idx, test_idx) in enumerate(fold_indices):
    fold = i
    print(f'Fold {fold+1}')
    print(f'\n test_index = {test_idx}')
    train_data, test_data = review_list[train_idx], review_list[test_idx]
    train_labels, test_labels = single_label_list[train_idx], single_label_list[test_idx]
    train_data, test_data, train_labels, test_labels = train_data.tolist(), test_data.tolist(), train_labels.tolist(), test_labels.tolist()

    # test_idxに含まれないIDに一致するインデックスを取得
    selected_indexes = [i for i, origin_id in enumerate(aug_origin_ID) if origin_id not in test_idx]

    # 上記のインデックスを使ってaug_review_listとaug_label_listをフィルタリング
    train_aug_review_list = [aug_review_list[i] for i in selected_indexes]
    train_aug_label_list = [aug_single_label_list[i] for i in selected_indexes]


    set_aug_review_list, set_aug_label_list = random_sample_sets(train_aug_review_list, train_aug_label_list, aug_review_num)
    print(f"train用拡張レビューの数 : {len(set_aug_review_list)}")
    print(f"train_num : {len(train_idx)}")
    print(f"test_num : {len(test_idx)}")

    #上のtrainデータにオーグメンテーションデータ追加
    train_data = train_data + set_aug_review_list
    train_labels = train_labels + set_aug_label_list
    if i == 4:
      train_texts, test_texts, train_labels, test_labels = train_data, test_data, train_labels, test_labels
      print(i)

Fold 1

 test_index = [ 11  12  18  19  23  29  34  40  41  43  47  52  54  59  61  68  69  77
  78  79  80  83  84  92  95  98 101 106 111 118 120 122 126 129 135 137
 144 148 151 172 173 178 180 189 191 192 196 198 200 203 215 220 221 223
 224 235 238 240 245 254 260 268 270 272 277 282 284 287 289 297 304 312
 318 328 335 341 342 350 353 356 357 372 376 379 382 387 399 406 407 411
 412 415 425 428 432 444 454 472 474 475 478 488 490 497 498 512 513 533
 535 540 547 552 554 563 570 573 580 582 590 594 595 617 624 628 635 645
 648 650 654 655 658 668 670 675 677 679 684 697 700 703 705 707 711 717
 733 737 750 751 753 758 760 761 762 765]
train用拡張レビューの数 : 256
train_num : 612
test_num : 154
Fold 2

 test_index = [  1   4  16  17  21  22  38  39  48  50  55  56  58  64  66  67  71  86
  90  96 104 121 123 128 133 136 143 145 149 152 157 164 179 187 190 193
 194 199 211 222 227 228 229 232 236 250 252 259 261 263 264 266 276 279
 280 281 288 298 303 314 315 319 324 331 333 338 346 347 35

In [ ]:
print(train_texts)
print(train_labels)

['暑さ対策\n思ったより大きかったです。\n音は静かです。充電は時間がかかりますが、電池の持ちは良いです。首にフィットして使い易いです。', '実際は、、、。\n2台買いました。とてもパワフルで良いのですが、風量1でも8〜9時間の連続使用が最長でした。風量3だとその半分くらいですかね。\n', 'コスパが良い\n登山時に使用。想像していたより軽量でフィット感もよく涼しい。', '静かな機種です。\n通勤に使ってます。\nフル充電、風量MAXで3時間使えます。\n電車内でも風量落とせば、音は気にならないです。欲を言えば、もっと風量が欲しいかな。', '暑さ対策で\n思ったより大きく\n少し重いけど、長い髪でも\n絡むこともなく\n良かったです', '\nリピートしたけど…違うブランドになっちゃって…比べてみたら…前の方が風が強くて…\n俺はまあまあかな', '良い\n暑い中外で使えば熱風だがないよりはマシ。\n室内で使う分には良し', '風力が強く満足です。\n肩こりの私には、ありがたい軽さです。お風呂上がりや、食事の支度など手放せません。体にフィットして安定感があります。風力が強くて満足ですが、その分、風の音が多少強いです。', '充電長持ちで良き\n風量1でしかほとんど使用しませんが充電は1日もつしじゅうぶんです。風量MAXは風音もうるさいしあまり使いません。\n※汗ふきシートなどで耳周りを拭いておくと、風でめっちゃ涼しくなるのでおすすめです。\n冷却プレートは直ぐに自分の体温でアチチになるので、気休めです笑\nあと2cmくらい後ろまで風口があれば最高でした。結局首の後ろが滝の汗。\n髪の毛が巻き込まれる事は無いですが、当たるとチリチリ言うのが少し不安になります。\n', '使いやすい\n操作が簡単で使いやすい。\n音が少し大きい。', '手持ち要らず\nファンの吹出口が横なので顔にはあたってますけど\u3000内側の丸いところからも風の吹出してくれたらより涼しいかと！改善の余地有るのでは?', '涼しい所で\n涼しい部屋や車内でMax運転すると凄く涼しいてわすよ。暑い屋外では使う気にはなれません。音はまぁまぁうるさいです。', 'いまいちです。\n屋外では使えません。ACの効いた部屋の中なら効果がありますが。\n私は田舎に住んでいますが近所の大都市（100万都

In [ ]:
#trainデータ
positive_taget_index = get_label_index(train_labels, 1)
positive_reviews, positive_labels = get_review_label_from_index(positive_taget_index, train_texts, train_labels)
negative_taget_index = get_label_index(train_labels, 0)
negative_reviews, negative_labels = get_review_label_from_index(negative_taget_index, train_texts, train_labels)
negative_reviews = negative_reviews[:len(positive_reviews)]
negative_labels = negative_labels[:len(positive_reviews)]
""""""
print(f'positive:{len(positive_labels)}')
print(f'negaive:{len(negative_labels)}')
train_texts = positive_reviews + negative_reviews
train_labels = positive_labels + negative_labels

positive:335
negaive:335


In [ ]:
# testデータ
test_positive_taget_index = get_label_index(test_labels, 1)
test_positive_reviews, test_positive_labels = get_review_label_from_index(test_positive_taget_index, test_texts, test_labels)
test_negative_taget_index = get_label_index(test_labels, 0)
test_negative_reviews, test_negative_labels = get_review_label_from_index(test_negative_taget_index, test_texts, test_labels)
test_negative_reviews = test_negative_reviews[:len(positive_reviews)]
test_negative_labels = test_negative_labels[:len(positive_reviews)]
""""""
print(f'positive:{len(positive_labels)}')
print(f'negaive:{len(negative_labels)}')
test_texts = test_positive_reviews + test_negative_reviews
test_labels = test_positive_labels + test_negative_labels

positive:335
negaive:335


# 隠れ層状態確認

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import numpy as np
import random
from sklearn.model_selection import StratifiedKFold

def get_hidden_states(texts, model, tokenizer):
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=128)
    with torch.no_grad():
        outputs = bert_sc(**inputs)
    hidden_states = outputs.hidden_states
    return hidden_states

train_hidden_states = get_hidden_states(train_texts, model, tokenizer)
test_hidden_states = get_hidden_states(test_texts, model, tokenizer)

print(f"1st: {len(train_hidden_states)=}") #tuple, (layers) = 13
print(f"1st: {len(train_hidden_states[0])=}") #tuple, (layers, samples) = 13, 200
print(f"1st: {len(train_hidden_states[0][0])=}") #tuple, (layers, samples, sequences) = 13, 200, 21
print(f"1st: {len(train_hidden_states[0][0][0])=}") #tuple, (layers, samples, sequences, embeddings) = 13, 200, 21, 768

1st: len(train_hidden_states)=13
1st: len(train_hidden_states[0])=670
1st: len(train_hidden_states[0][0])=128
1st: len(train_hidden_states[0][0][0])=768


In [ ]:
# Flatten hidden states (シーケンス長異なると困るので、トークン単位での平均を求める)
# hは (samples, sequences, embeddings)。
# dim=1 で (dim=0, dim=1, dim=2) のインデックス指定になり、sequencesの平均を求めている
train_hidden_states = [torch.mean(h, dim=1).numpy() for h in train_hidden_states]
test_hidden_states = [torch.mean(h, dim=1).numpy() for h in test_hidden_states]
print(f"flatten: {len(train_hidden_states)=}") #list (layers) = 13
print(f"flatten: {len(train_hidden_states[0])=}") #list (layers, samples) = 13, 200
print(f"flatten: {len(train_hidden_states[0][0])=}") #list (layers, samples, embeddings) = 13, 200, 768

flatten: len(train_hidden_states)=13
flatten: len(train_hidden_states[0])=670
flatten: len(train_hidden_states[0][0])=768


In [ ]:
# reshapeするためnp.arrayに変換 (layers, samples, embeddings)
train_hidden_states = np.stack(train_hidden_states, axis=0)
test_hidden_states = np.stack(test_hidden_states, axis=0)
print(f"stacked: {train_hidden_states.shape=}")

stacked: train_hidden_states.shape=(13, 670, 768)


In [ ]:
# Reshape to ensure the correct dimensions
#train_hidden_states = np.array([h for h in train_hidden_states])
#test_hidden_states = np.array([h for h in test_hidden_states])
#print(f"reshaped: {train_hidden_states.shape=}")

# train_hidden_statesの要素順を入れ替え。
# np.transpose()の(1,0,2)は、
#    元の1次元目(samples)を0次元目に移動。
#    元の0次元目(layers)を1次元目に移動。
#    元の2次元目(embeddings)を2次元目にそのまま保持。
# 元の順序: (layers=0, samples=1, embeddings=2)
#   新順序: (samples=0, layers=1, embeddings=2)
train_hidden_states = np.transpose(train_hidden_states, (1, 0, 2))
test_hidden_states = np.transpose(test_hidden_states, (1, 0, 2))
print(f"transposed: {train_hidden_states.shape=}")

transposed: train_hidden_states.shape=(670, 13, 768)


#プロービング

In [ ]:
#train_hidden_states内容確認
print(train_hidden_states[1, 3, : 100])

[ 0.22245015 -0.32776535 -0.11370103  0.14726105  0.02010678  0.2399962
 -0.15171705 -0.02905483 -0.36417165 -0.16311517 -0.1281017   0.15227512
  0.072116    0.41422215  0.14644572 -0.40262404  0.06073534 -0.08312308
  0.13959092  0.19190724  0.02741019 -0.09381057  0.05114596  0.25573167
  0.03625013  0.13994944 -0.03576023 -0.12732126  0.00989313  0.24368955
  0.01499781  0.08254039  0.10376631  0.13551328  0.26235786 -0.11367832
 -0.1678944   0.10443611  0.14724748 -0.08147392 -0.12072039  0.0185164
  0.06074066  0.32227013  0.00164815 -0.4045806  -0.4298886   0.21759728
 -0.03598416 -0.00165699  0.16672489 -0.11399734  0.04339682  0.13028455
 -0.39148504 -0.13767423  0.12239185  0.10239403 -0.07097757 -0.33746704
 -0.03651489  0.19025627 -0.13339753 -0.12819435  0.09621064 -0.18097976
 -0.2516767  -0.49467182 -0.6087493  -0.24779363  0.04216826  0.41975507
 -0.15823072 -0.13147105  0.05123618 -0.1491563  -0.03077831  0.00987973
  0.1441226   0.35911295  0.20717119 -0.06771696  0.1

In [ ]:
# Step 3: プロービング
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
accuracies, confusion_matrixs, precision_scores, recall_scores = [], [], [], []

for layer in range(train_hidden_states.shape[1]):
    X_train = train_hidden_states[:, layer, :]
    #print(f"{X_train.shape=}")
    X_test = test_hidden_states[:, layer, :]

    clf = LogisticRegression(max_iter=1000)
    clf.fit(X_train, train_labels)

    predictions = clf.predict(X_test)
    accuracy = accuracy_score(test_labels, predictions)
    confusion = confusion_matrix(test_labels, predictions)
    precision = precision_score(test_labels, predictions)
    recall = recall_score(test_labels, predictions)

    accuracies.append(accuracy)
    confusion_matrixs.append(confusion)
    precision_scores.append(precision)
    recall_scores.append(recall)
    #print(f"Layer {layer + 1}: Test Accuracy = {accuracy:.4f}")

# 最終的な分類精度の表示
print("Layer 0 Accuracy Recall Precision")
for layer, accuracy in enumerate(accuracies, 1):
    print(f"Layer {layer} {accuracy:.4f} {recall_scores[layer-1]:.4f} {precision_scores[layer-1]:.4f}")

for layer, confusion in enumerate(confusion_matrixs, 1):
    print(f"Layer {layer}: Confusion Matrix\n {confusion}")

Layer 0 Accuracy Recall Precision
Layer 1 0.8627 0.4000 0.4706
Layer 2 0.8235 0.4000 0.3478
Layer 3 0.8366 0.4500 0.3913
Layer 4 0.8497 0.4000 0.4211
Layer 5 0.8497 0.4500 0.4286
Layer 6 0.8301 0.5000 0.3846
Layer 7 0.8235 0.4000 0.3478
Layer 8 0.8366 0.5500 0.4074
Layer 9 0.8562 0.4500 0.4500
Layer 10 0.8562 0.6000 0.4615
Layer 11 0.8366 0.6000 0.4138
Layer 12 0.8431 0.5500 0.4231
Layer 13 0.8431 0.5000 0.4167
Layer 1: Confusion Matrix
 [[124   9]
 [ 12   8]]
Layer 2: Confusion Matrix
 [[118  15]
 [ 12   8]]
Layer 3: Confusion Matrix
 [[119  14]
 [ 11   9]]
Layer 4: Confusion Matrix
 [[122  11]
 [ 12   8]]
Layer 5: Confusion Matrix
 [[121  12]
 [ 11   9]]
Layer 6: Confusion Matrix
 [[117  16]
 [ 10  10]]
Layer 7: Confusion Matrix
 [[118  15]
 [ 12   8]]
Layer 8: Confusion Matrix
 [[117  16]
 [  9  11]]
Layer 9: Confusion Matrix
 [[122  11]
 [ 11   9]]
Layer 10: Confusion Matrix
 [[119  14]
 [  8  12]]
Layer 11: Confusion Matrix
 [[116  17]
 [  8  12]]
Layer 12: Confusion Matrix
 [[118

# 制御タスク

In [ ]:
# シャッフルしたラベルを用意
import random
control_labels = [1] * len(positive_labels) + [0] * len(negative_labels)
random.shuffle(control_labels)
control_labels[:10]

[0, 1, 1, 1, 0, 1, 1, 1, 0, 1]

In [ ]:
print(len(control_labels))

670


In [ ]:
control_train_labels = control_labels[:len(train_labels)]
control_test_labels = control_labels[:len(test_labels)]

In [ ]:
len(get_label_index(test_labels, 0))

133

In [ ]:
import numpy as np


# Step 3: プロービング
#施行回数
act_num = 100
all_accuracies = []
for i in range(act_num):
  accuracies = []

  control_labels = [1] * len(positive_labels) + [0] * len(negative_labels)
  random.shuffle(control_labels)
  control_labels[:10]

  control_train_labels = control_labels[:len(train_labels)]
  control_test_labels = control_labels[:len(test_labels)]

  for layer in range(train_hidden_states.shape[1]):
      X_train = train_hidden_states[:, layer, :]
      #print(f"{X_train.shape=}")
      X_test = test_hidden_states[:, layer, :]

      clf = LogisticRegression(max_iter=1000,random_state=i)
      clf.fit(X_train, control_train_labels)

      predictions = clf.predict(X_test)
      accuracy = accuracy_score(control_test_labels, predictions)
      accuracies.append(accuracy)
      #print(f"Layer {layer + 1}: Control Test Accuracy = {accuracy:.4f}")

  all_accuracies.append(accuracies)

"""
# 最終的な分類精度の表示
for layer, accuracy in enumerate(accuracies, 1):
    print(f"Layer {layer} Control Test Accuracy = {accuracy:.4f}")
"""
# 各層の平均を計算
average_accuracies = np.mean(all_accuracies, axis=0)

# 各層の平均精度を出力
for layer, avg_accuracy in enumerate(average_accuracies, 1):
    print(f"Layer {layer} Average Control Test Accuracy = {avg_accuracy:.4f}")

Layer 1 Average Control Test Accuracy = 0.4995
Layer 2 Average Control Test Accuracy = 0.5022
Layer 3 Average Control Test Accuracy = 0.4992
Layer 4 Average Control Test Accuracy = 0.4971
Layer 5 Average Control Test Accuracy = 0.4954
Layer 6 Average Control Test Accuracy = 0.4956
Layer 7 Average Control Test Accuracy = 0.4987
Layer 8 Average Control Test Accuracy = 0.4999
Layer 9 Average Control Test Accuracy = 0.5044
Layer 10 Average Control Test Accuracy = 0.4992
Layer 11 Average Control Test Accuracy = 0.5034
Layer 12 Average Control Test Accuracy = 0.4959
Layer 13 Average Control Test Accuracy = 0.4962


In [ ]:
from scipy.stats import ttest_1samp
import numpy as np

# 各層の精度を格納したリスト (10回施行した場合の結果)
# 例: all_accuracies は shape (試行回数, 層数) の2Dリスト/配列
all_accuracies = np.array(all_accuracies)  # 施行回数 x 層数

# 各層ごとに t 検定を実施
t_values, p_values = [], []
for layer_idx in range(all_accuracies.shape[1]):  # 層ごとのループ
    layer_accuracies = all_accuracies[:, layer_idx]  # その層の施行ごとの精度
    t_stat, p_value = ttest_1samp(layer_accuracies, 0.5)  # t検定 (期待値 = 0.5)
    t_values.append(t_stat)
    p_values.append(p_value)

# 結果を表示
print("Layer | t-value | p-value")
for layer, (t_val, p_val) in enumerate(zip(t_values, p_values), 1):
    print(f"Layer {layer:>2} | {t_val:>7.4f} | {p_val:>7.4f}")

# 有意水準 (例: 5%) を超える場合の判断
alpha = 0.05
for layer, p_val in enumerate(p_values, 1):
    if p_val < alpha:
        print(f"Layer {layer} は p < {alpha} で統計的に有意です。")
    else:
        print(f"Layer {layer} は p >= {alpha} で統計的に有意ではありません。")


Layer | t-value | p-value
Layer  1 | -0.1275 |  0.8988
Layer  2 |  0.5226 |  0.6024
Layer  3 | -0.2199 |  0.8264
Layer  4 | -0.7013 |  0.4848
Layer  5 | -1.2914 |  0.1996
Layer  6 | -1.2766 |  0.2047
Layer  7 | -0.3423 |  0.7328
Layer  8 | -0.0157 |  0.9875
Layer  9 |  1.1952 |  0.2349
Layer 10 | -0.2007 |  0.8414
Layer 11 |  0.7545 |  0.4523
Layer 12 | -0.9424 |  0.3483
Layer 13 | -0.8755 |  0.3834
Layer 1 は p >= 0.05 で統計的に有意ではありません。
Layer 2 は p >= 0.05 で統計的に有意ではありません。
Layer 3 は p >= 0.05 で統計的に有意ではありません。
Layer 4 は p >= 0.05 で統計的に有意ではありません。
Layer 5 は p >= 0.05 で統計的に有意ではありません。
Layer 6 は p >= 0.05 で統計的に有意ではありません。
Layer 7 は p >= 0.05 で統計的に有意ではありません。
Layer 8 は p >= 0.05 で統計的に有意ではありません。
Layer 9 は p >= 0.05 で統計的に有意ではありません。
Layer 10 は p >= 0.05 で統計的に有意ではありません。
Layer 11 は p >= 0.05 で統計的に有意ではありません。
Layer 12 は p >= 0.05 で統計的に有意ではありません。
Layer 13 は p >= 0.05 で統計的に有意ではありません。
